# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Optimizing Self-Driving

Notebook 3 - Modeling (part 2)

# Imports

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Lambda, Conv2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.losses import Huber
from utils import INPUT_SHAPE, batch_generator

# Import Data

In [41]:
# read in data, labeling columns
df = pd.read_csv('../data/data-anthony/driving_log.csv', names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

# Data Processing

In [42]:
# replace local file paths to cloud file paths
df.center = df.center.str.replace('/Users/anthonysmacbook/DSI/week_7/How_to_simulate_a_self_driving_car/data', '../data-anthony')
df.left = df.left.str.replace('/Users/anthonysmacbook/DSI/week_7/How_to_simulate_a_self_driving_car/data', '../data-anthony')
df.right = df.right.str.replace('/Users/anthonysmacbook/DSI/week_7/How_to_simulate_a_self_driving_car/data', '../data-anthony')

In [43]:
# construct feature matrix and target vector
X = df[['center', 'left', 'right']].values
y = df['steering'].values

In [44]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

Note that although not shown here, the images go through a lot of processing before they are fed into the neural network. The image processing is built into the generator used in the accompanying utils.py file. This allows us to simultaneously train our model using our GPU while our data is being prepared / images are being processed on the CPU. Additionally, the generator allows us to repeatedly train our model on one lap of data.

# Modeling

In [45]:
# convolutional neural network
model = Sequential()
model.add(Lambda(lambda x: x/127.5-1.0, input_shape=INPUT_SHAPE)) # normalize image from -1 to 1
model.add(Conv2D(filters=24, kernel_size=(5, 5), activation='elu', strides=2))
model.add(Conv2D(filters=36, kernel_size=(5, 5), activation='elu', strides=2))
model.add(Conv2D(filters=48, kernel_size=(5, 5), activation='elu', strides=2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='elu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='elu'))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))

In [46]:
# checkpoints
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 mode='auto')

In [47]:
# custom loss function, inputs y_true and y_pred and outputs loss.
# Includes mean squared error plus alpha and beta terms.
# alpha penalizes frequent swerves (high frequency swerves)
# beta penalizes large steering values (big swerves)
def custom_loss(y_true, y_pred):
    mse = K.sum(K.square(y_true - y_pred)) / K.cast(len(y_true), 'float32')
    
    sign_change_count = 0
    
    for i in range(1, len(y_pred)):
        if y_pred[i] == 0 or y_pred[i-1] == 0:
            continue
        elif y_pred[i] / y_pred[i-1] < 0:
            sign_change_count += 1
                
    alpha = 0.1
    beta = 0.05
    
    y_pred_sum = K.sum(K.square(y_pred))
    
    loss = mse + (K.cast(alpha, 'float32') * K.cast(sign_change_count, 'float32')) + (beta * y_pred_sum)
    return loss

In [49]:
huber = Huber(delta=1.0)

In [50]:
# compile model with custom loss function and optimizer
model.compile(loss = huber, optimizer = Adam(lr=1.0e-4))

In [51]:
# fit with generator
# generator allows us to repeatedly train on one lap of data.
model.fit(x=batch_generator('../data/data-anthony', X_train, y_train, 40, True),
                        steps_per_epoch=20000,
                        epochs=3,
                        max_queue_size=1,
                        validation_data=batch_generator('../data/data-anthony', X_test, y_test, 40, False),
                        validation_steps=len(X_test),
                        callbacks=[checkpoint],
                        verbose=1)

Epoch 1/3
20000/20000 [==============================] - 2280s 114ms/step - loss: 0.0075 - val_loss: 0.0101
Epoch 2/3
20000/20000 [==============================] - 2352s 118ms/step - loss: 0.0053 - val_loss: 0.0102
Epoch 3/3
20000/20000 [==============================] - 2351s 118ms/step - loss: 0.0046 - val_loss: 0.0102
